In [1]:
import requests
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import os
from requests.auth import HTTPBasicAuth
import json
import time
import logging
import sys

In [2]:
import os

In [3]:
import df_construct

In [4]:
#os.chdir(r"D:\Textkernel")

In [5]:
YEAR_MONTH = '2020-09'

In [6]:
log_format = (
    '[%(asctime)s] %(levelname)-8s %(name)-12s %(message)s')

logging.basicConfig(
    level=logging.DEBUG,
    format=log_format,
    # Declare handlers
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('debug_%s.log'%YEAR_MONTH),
    ]
)
# Define your own logger name
logger = logging.getLogger("my_logger_%s"%YEAR_MONTH)

### Diese Funktion iteriert für einen gegebenen Monat über alle (bekannten) Regionen, verwendet diese als Input für die "get_month_region" Funktionen, vergleich die Anzahl der STEAs und speichert das Dataframe als JSON

In [7]:
def get_complete_month_2(month_select):
    
    
    ergebnis = DataFrame()
    #Achtung es gibt eine 17. Region (UN=Unbekannt)
    
    ##Achtung ! Eine Region ohne Einträge in einem ganzen Monat (vor aallem UN ) scheint Fehler zu verursachen 
    #region = ['BE','SL','RP','BB','MV','SH','ST','SN','HH','BW','NI','TH','UN','HB','NW','BY','HE' ]
    
    region = [ 'NW','BE','MV','SH','ST','SN','HH','NI','TH','HB','SL','HE','BB','BW','RP','BY' ]
    
    ##ACTHUnG TEST!!
    #region = ['BE','SL','BW','RP' ]
 
    

    counter = 0
    region_counter = 0
    agg2 = df_construct.aggregate_month(month_select)
    
    logger.info("Anzahl Regionen  "+str(len(region)))

    for r in region:
        # record start time
        start = time.time()
        region_counter+= 1
        
        filename = get_filename(month_select,r)
        if os.path.exists(filename+'.json'):
            logger.debug ('%s: file %s already exists.'%(region_counter,filename))
            continue
        logger.debug ('%s: processing %s ...'%(region_counter,filename))
            
        ##NEU
        df_construct.toobig = False
        
        #Neue Retry Funktion
        ergebnis_r = df_construct.get_month_region_retry(r,month_select)
        
        ##Neu
        
        agg1 = df_construct.aggregate_month_region(r,month_select)
        
        ##Wie viele Anzeigen runtergeladen
        counter = counter + len(ergebnis_r)
        
        ##NEU nach JSON per Monat/Region
        
        #NEU!
        #filename = get_filename(month_select,r)
        write_json(filename,ergebnis_r)
        
        ##NEU! Für den 2ten Teil des Programms muss der Filename verfügbar sein
        
        files.append(filename)
        
        ## Die Daten über die heruntergeladenen STEAS 
        mdata = []
        
        ##Monat/Jahr Region und heruntergeladene Steas
        #mdata.append(month_select)
        
        #NEU!!!
        jahr = month_select.split('-')[0]
        monat= month_select.split('-')[1]
        
        mdata.append(jahr)
        mdata.append(monat)
        
        ###
        
        mdata.append(r)
        mdata.append(len(ergebnis_r))
        
        ## Steas für diesen Monat/Jahr/Region die aktuelle Anzahl der Steas in allen Downloads und die Anzahl der Steas für
        ## den gesamten Monat des Jahres (mit den unbekannten Regionen)
        mdata.append(agg1)
        mdata.append(counter)
        mdata.append(agg2)
        
        ##Und der Dateiname für später
        mdata.append(filename)
        
        ##TEST die globale Variable für zu große Tage
        mdata.append(df_construct.toobig)
        
        ##Tupel wird der globalen Variable hinzugefügt
        metadata.append(mdata)
        
        ## Speicher wieder freigeben 
        mdata = None
        ergebnis_r = None
    
        # record execution time
        executionTime = (time.time () - start) 

        # print the difference between start
        # and end time in milli. secs
        logger.debug("The time of execution of PROCESSING df for region %s : %s"%(r, sec_to_hms(int(executionTime))))
      
    
    logger.debug("Programm fertig")    

In [8]:
##NEU Funktion soll weiter helfen die Dateinamen zu vereinheitlichen 
def get_filename(month,region):
    
    filename = "TK_"+month+"_"+region
    
    return filename

In [9]:
## Die Ausgabe ist für die json --> DB Funktion gedacht 
def get_month(filename):
    
    month = filename[3:10]
  
    return month

In [10]:
def sec_to_hms(seconds):
    h = seconds // 3600
    m = seconds % 3600 // 60
    s = seconds % 3600 % 60
    return f'{h:02d}:{m:02d}:{s:02d}'

In [11]:
##NEUE Parameter
def write_json(filename,df):
    # record start time
    start = time.time()
    
    #Das macht die Funtion nun nicht mehr selbst
    #filename = month+region
    
    
    logger.debug("Schreibe als json:  "+filename)
    print("Schreibe als json:  "+filename,file=text_file)
    
    
    ##Sollte Force_Ascii nicht true sein: NEIN für Textkernel und Co brauch man Latin-1 encoding NICHT utf-8
    df.to_json(filename+".json", orient="records", force_ascii = False)
    
    ##df.to_json(filename+".json", orient="records", force_ascii = True)
    
    logger.debug("JSON Datei erstellt")
    print("JSON Datei erstellt",file=text_file)
    
    # record execution time
    executionTime = (time.time () - start) 

    # print the difference between start
    # and end time in milli. secs
    logger.debug("The time of execution of writing on json is : %s",
          sec_to_hms(int(executionTime)))

In [12]:
##Der Februar ist der Monat mit den meisten Einträgen
### Format YYYY-MM
def testlauf():
    
    months = [YEAR_MONTH] #,'2022-05','2022-06','2022-07','2022-08','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12','2022-01','2022-02','2022-03','2022-04']
    
    for m in months:
        get_complete_month_2(m)
        


#### Startpunkt für das Progrtamm (Teil 1) , initialisiert eine Variable die den Zugriff für die Log Datei enthält sowie zwei Arrays die global vorliegen

In [13]:
## Hier starten

##Dieser Array wird Stück für Stück befüllt auch wenn testlauf() abbricht sind die Daten nicht weg!
## Diese Variable bleibt vorläufig erhalten das sie Stück für Stück befüllt wird 

files = []

## Diese Daten müssen auch in  einem Array erfasst werden und erst dann in ein dataframe und dann json umgewandelt werden
metadata = []


##Globale Variable zur Erfassung von Tagen mit 10000+ Einträgen
df_construct.toobig = False

text_file = open(("txt_kernel_download_log_01092022_neu_skills_%s.txt"%YEAR_MONTH), "a")

#Kopie für Modul TEST
df_construct.text_file=text_file


testlauf()


print(files)

## Erstellung der JSON mit den Daten des Vorgangs
df_meta = pd.DataFrame(metadata, columns =['Jahr','Monat', 'Region','SteasDownload','SteasMonatRegion','SteasDownloadTotal','SteasMonatTotal','Dateiname','TagZuGross']) 
df_meta

write_json('testlauf1_%s'%YEAR_MONTH,df_meta)


text_file.close()

[2023-03-03 14:20:14,689] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:20:15,495] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2020-09-01__2020-09-01%7C%7C%2B1M-1d%0A%0A HTTP/1.1" 200 None
[2023-03-03 14:20:15,498] INFO     my_logger_2020-09 Anzahl Regionen  16
[2023-03-03 14:20:15,499] DEBUG    my_logger_2020-09 1: processing TK_2020-09_NW ...
[2023-03-03 14:20:18,513] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:20:19,137] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2020-09-01__2020-09-01%7C%7C%2B1M-1d%0A%0A&region=NW HTTP/1.1" 200 None
[2023-03-03 14:20:22,148] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:20:26,907] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-09-01&_

[2023-03-03 14:24:42,446] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-09-27&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:24:47,890] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:24:50,449] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-09-28&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:24:55,918] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:25:01,071] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-09-29&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:25:08,335] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:25:13,881] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-09-30&_limit=10000&_

[2023-03-03 14:27:52,692] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-09-21&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:27:56,505] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:27:58,921] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-09-22&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:28:03,761] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:28:05,892] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-09-23&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:28:10,132] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:28:12,836] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-09-24&_limit=10000&_

[2023-03-03 14:30:13,027] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-09-15&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:30:16,339] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:30:17,250] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-09-16&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:30:20,582] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:30:21,532] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-09-17&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:30:25,026] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:30:26,086] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-09-18&_limit=10000&_

[2023-03-03 14:32:10,423] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-09-09&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:32:14,126] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:32:15,559] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-09-10&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:32:19,144] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:32:20,320] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-09-11&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:32:24,000] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:32:24,873] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-09-12&_limit=10000&_

...

[2023-03-03 14:57:39,933] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BB&date=2020-09-13&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:57:43,163] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:57:44,060] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BB&date=2020-09-14&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:57:47,467] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:57:48,535] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BB&date=2020-09-15&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 14:57:52,052] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 14:57:53,213] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BB&date=2020-09-16&_limit=10000&_

[2023-03-03 15:00:03,640] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2020-09-07&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:00:08,134] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:00:11,253] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2020-09-08&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:00:16,363] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:00:20,476] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2020-09-09&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:00:26,111] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:00:29,078] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2020-09-10&_limit=10000&_

[2023-03-03 15:03:53,846] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=RP&date=2020-09-01&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:03:57,283] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:03:58,725] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=RP&date=2020-09-02&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:04:02,451] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:04:03,593] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=RP&date=2020-09-03&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:04:07,181] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:04:08,507] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=RP&date=2020-09-04&_limit=10000&_

[2023-03-03 15:06:24,701] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=RP&date=2020-09-30&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:06:28,547] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:06:29,075] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=RP&date=2020-09-31&_limit=10000&_labels=0 HTTP/1.1" 500 None
[2023-03-03 15:06:32,532] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:06:33,095] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2020-09-01__2020-09-01%7C%7C%2B1M-1d%0A%0A&region=RP HTTP/1.1" 200 None
[2023-03-03 15:06:33,099] DEBUG    my_logger_2020-09 Schreibe als json:  TK_2020-09_RP
[2023-03-03 15:06:36,104] DEBUG    my_logger_2020-09 JSON Datei erstellt
[2023-03-03 15:06:36,108] DEBUG    my_logger_2020-09 The time of execution of 

[2023-03-03 15:10:13,054] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BY&date=2020-09-24&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:10:19,537] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:10:23,453] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BY&date=2020-09-25&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:10:29,715] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:10:33,483] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BY&date=2020-09-26&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:10:39,842] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:10:41,917] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BY&date=2020-09-27&_limit=10000&_

In [14]:
text_file.close()

In [15]:

text_file = open("txt_kernel_LOG.txt", "a")

#Kopie für Modul TEST
df_construct.text_file=text_file

In [16]:
test=  df_construct.get_month_region_retry('BW','2022-08')
print(df_construct.toobig)

[2023-03-03 15:11:44,697] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:11:45,408] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2022-08-01__2022-08-01%7C%7C%2B1M-1d%0A%0A&region=BW HTTP/1.1" 200 None
[2023-03-03 15:11:48,433] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:11:53,918] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-01&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:11:59,169] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:12:07,038] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-02&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:12:12,570] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15

[2023-03-03 15:17:27,698] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:17:32,449] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-29&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:17:38,872] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:17:45,909] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-30&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:17:52,865] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:18:00,755] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-31&_limit=10000&_labels=0 HTTP/1.1" 200 None
False


#### Bibliotheken und globale Variablen für die Befüllung der DB 

#### Hier findet der 2.Teil statt in dem der Transfer in die DB getestet wird

In [17]:

#x = pd.read_json("df_error.json")

x = pd.read_json("TK_2021-03_BE.json",orient="records", encoding="latin-1")

ValueError: Expected object or value

In [ ]:
x['it_skills']

In [ ]:
index = x.index
columns = x.columns
columns

In [ ]:
#x[x['it_skill_terms']=='[LabView]']
#x

In [ ]:
#x["Monat"]= x["Dateiname"].apply(lambda x: x[-5:-3]) Nicht mehr nötig

In [ ]:
files = x['Dateiname'].tolist() ##Abgeöndert

In [ ]:
files

In [ ]:
##NEUE Tests

text_file = open("txt_kernel_test_NEU.txt", "a")
#test = get_month_region_retry('2020-04','BW')

#Kopie für Modul TEST
df_construct.text_file=text_file


##Wichtig !  Region + Monat mir Tag über 10000 Einträgen
#test=  df_construct.get_month_region_retry('BW','2020-01')

#test=  df_construct.get_month_region_retry('BW','2020-04')
test=  df_construct.get_month_region_retry('SL','2018-03')

#test=  df_construct.get_month_region_retry('2020-04','BW')
#construct_day('2020-04','BE','23')

In [ ]:
test
len(test.job_id.unique())

In [ ]:
text_file = open("agg_test.txt", "a")
#test = get_month_region_retry('2020-04','BW')

#Kopie für Modul TEST
df_construct.text_file=text_file
x = df_construct.aggregate_month_region("SL","2018-03")


In [ ]:
print(x)

In [ ]:
df_input = pd.read_json('TK_2020-08_BE.json',encoding="latin-1")

In [ ]:
df_input